Careless responder feature engineering and analysis (inital focus on psych_flex survey only)

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt
from numpy import median

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/data_final_combined.pkl'

In [ ]:
#read in csv containing participant info
data = pd.read_pickle(data_path)
data = pd.DataFrame(data = data)
#data_PF.reset_index(inplace=True)

print('Original data shape:\n', data.shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['ParticipantID'].unique().shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['MitreID'].unique().shape, '\n')
#how much missing data is there?
print('Original data missing value counts:\n', data.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data data types:\n', data.info(), '\n')

In [ ]:
data['survey_type'].value_counts()

In [ ]:
#should be 71
len(data['wave_study_day'].unique())

### notes for CR features for Psych Flex

Should have answered every question

Longstring
- Legitimate longstrings of  ≥ 8 are unlikely for response “5”
    - make column with longest string
    - make column with number that longest string consisted of

Semantic consistency
- Legitimate scores of pf_mgt=5 are almost impossible

Semantic antonyms
- Not applicable

Semantic synonyms 
- not applicable 


In [ ]:
#split off completed PF and related columns
psych_flex_only = data[(data['survey_type'] == 'psych_flex') & (data['completed'] == 1.0)]
psych_flex_only = psych_flex_only[['index', 'MitreID', 'ParticipantID', 'PrimaryUnit', 'SmartPhone',
       'Sex', 'Shift', 'Wave', 'survey_id', 'survey_type', 'date_time',
       'date', 'time', 'Timesent', 'completed', 'start_delay',
       'time_to_complete', 'activity_num', 'location_num',
       'activity', 'pf_03', 'pf_04', 'pf_05', 'pf_06',
       'pf_07', 'pf_08', 'pf_09', 'pf_10', 'pf_11', 'pf_12', 'pf_13',
       'pf_14', 'pf_15', 'pf_mgt', 'exp_0', 'exp_1', 'exp_2', 'exp_3',
       'exp_4', 'exp_5', 'exp_6', 'exp_7', 'exp_8', 'exp_9', 'exp_10',
       'exp_11', 'exp_12', 'exp_13', 'exp_neg', 'exp_pos', 'exp_neut',
       'wave_study_date', 'wave_study_day',
       'survey_count', 'day_of_week', 'working', 'at_work']]

psych_flex_only.shape

In [ ]:
#create study date bins
psych_flex_only['wave_study_date_bin'] = pd.cut(psych_flex_only['wave_study_day'], 5)
psych_flex_only['wave_study_date_bin'].value_counts()

In [ ]:
#create feature that is count of missing values from survey
psych_flex_only.loc[psych_flex_only.loc[:, 'activity':'pf_mgt'].isnull().sum(axis=1) > 0, 'missing_values'] = psych_flex_only.loc[:, 'activity':'pf_mgt'].isnull().sum(axis=1)
psych_flex_only.loc[psych_flex_only.loc[:, 'activity':'pf_mgt'].isnull().sum(axis=1) == 0, 'missing_values'] = 0.0

In [ ]:
#create feature that is number of surveys that were not completed

part_id = psych_flex_only['ParticipantID'].unique()

for participant in part_id:
    missing = 49 - psych_flex_only[psych_flex_only['ParticipantID'] == participant]['survey_count'].max()
    psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant), 'uncompleted'] = missing

In [ ]:
#create feature that is number of surveys that were not completed within wave study date bin

part_id = psych_flex_only['ParticipantID'].unique()
day_bins = psych_flex_only['wave_study_date_bin'].unique()

for participant in part_id:
    for unique_bin in day_bins:
        missing = 10 - psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == unique_bin)].shape[0]
        if missing < 0:
            missing = 0
        psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == unique_bin), 'uncompleted_day_bin'] = missing

In [ ]:
#long string analysis (e.g. max length of same number answered for pf_03:pf_15)
#create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

max_strings = []
max_answers = []

for index, row in psych_flex_only.iterrows():
    
    groups = groupby(row['pf_03':'pf_15'])
    result = [(label, sum(1 for _ in group)) for label, group in groups]

    max_pair = max(result, key=lambda x:x[1])
    max_string_length = max_pair[1]
    max_answer = max_pair[0]

    max_strings.append(max_string_length)
    
    max_answers.append(max_answer)

psych_flex_only['longest_string_count'] = max_strings
psych_flex_only['longest_string_answer'] = max_answers

In [ ]:
#create feature that is ratio of average longstring during first day bin to average longstring during last time bin

part_id = psych_flex_only['ParticipantID'].unique()
day_bins = psych_flex_only['wave_study_date_bin'].unique()

for participant in part_id:
    string_1 = psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == day_bins[0])]['longest_string_count'].mean() 
    string_2 = psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == day_bins[-1])]['longest_string_count'].mean()

    psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant), 'longest_string_ratio'] = string_2 / string_1

In [ ]:
#create feature that is ratio of average longstring during current bin to average longstring during last time bin

part_id = psych_flex_only['ParticipantID'].unique()
day_bins = psych_flex_only['wave_study_date_bin'].unique()

for participant in part_id:
    for unique_bin in day_bins:
        string_1 = psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == unique_bin)]['longest_string_count'].mean() 
        string_2 = psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == day_bins[-1])]['longest_string_count'].mean()

        psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == unique_bin), 'longest_string_ratio_bin'] = string_2 / string_1

In [ ]:
#create feature that is ratio of number of longstrings over 7 to total number of surveys taken

part_id = psych_flex_only['ParticipantID'].unique()

for participant in part_id:
    count_over_7 = psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['longest_string_count'] >= 8)].shape[0]
    
    psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant), 'ls_7_ratio'] = count_over_7 / psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant)].shape[0]

In [ ]:
#create feature that is ratio of number of longstrings over 7 during current bin to total number of surveys taken for that bin

part_id = psych_flex_only['ParticipantID'].unique()
day_bins = psych_flex_only['wave_study_date_bin'].unique()

for participant in part_id:
    for unique_bin in day_bins:
        count_over_7 = psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) 
                                           & (psych_flex_only['longest_string_count'] >= 8) 
                                           & (psych_flex_only['wave_study_date_bin'] == unique_bin)].shape[0]
        try:
            psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == unique_bin), 'ls_7_ratio_bin'] = count_over_7 / psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == unique_bin)].shape[0]
        except:
            psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant) & (psych_flex_only['wave_study_date_bin'] == unique_bin), 'ls_7_ratio_bin'] = np.nan 

In [ ]:
#create feature that is binary for if time_to_complete is very short or long (0 is neither)
bad_times_low = []
bad_times_high = []

for index, row in psych_flex_only.iterrows():
    
    if (row['time_to_complete'] <= 20): 
        bad_times_low.append(1)
    else:
        bad_times_low.append(0)
    
    if (row['time_to_complete'] > 1200): 
        bad_times_high.append(1)
    else:
        bad_times_high.append(0)

psych_flex_only['bad_times_low'] = bad_times_low
psych_flex_only['bad_times_high'] = bad_times_high

In [ ]:
psych_flex_CR = psych_flex_only[['MitreID', 'ParticipantID', 'PrimaryUnit', 'SmartPhone',
       'Sex', 'Shift', 'Wave', 'start_delay',
       'time_to_complete', 'activity_num', 'pf_mgt',
       'exp_neg', 'exp_pos', 'exp_neut',
       'wave_study_day', 'survey_count', 'day_of_week', 'working',
       'wave_study_date_bin', 'missing_values', 'uncompleted',
       'uncompleted_day_bin', 'longest_string_count',
       'longest_string_answer', 'longest_string_ratio',
       'longest_string_ratio_bin', 'bad_times_low', 'bad_times_high', 'ls_7_ratio', 'ls_7_ratio_bin']]

psych_flex_CR.head()

In [ ]:
path_demog_prepost = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/Demog, PRE, PST survey composites.csv'

In [ ]:
#read in data_demog_prepost, contains demographic information and pre/post questionaires
data_demog_prepost = pd.read_csv(path_demog_prepost)
data_demog_prepost = pd.DataFrame(data = data_demog_prepost)
print('Original demog_prepost shape:\n', data_demog_prepost.shape, '\n')
#replace blnaks with nans
data_demog_prepost = data_demog_prepost.replace(' ', np.nan)
#ensure no replicate ID (eg one row per participant in study)
print('Original demog_prepost unique IDs:\n', data_demog_prepost['ID'].unique().shape, '\n')
print('Original demog_prepost missing value couts:\n', data_demog_prepost.isnull().sum(), '\n')
print('Original demog_prepost data types:\n', data_demog_prepost.info(), '\n')
data_demog_prepost.head()

In [ ]:
#add the information contained in data_demog_prepost to psych_flex_CR
#first create new data table of data_demog_prepost that contains the same number of rows for each participant in that is in psych_flex_CR
#(e.g. replicate data_demog_prepost so same length as psych_flex_CR for each participant)

participants = psych_flex_CR['MitreID'].unique()

psych_flex_CR_demog = pd.DataFrame()

for part in participants:
    df_part_long = pd.concat([data_demog_prepost[data_demog_prepost['ID'] == part]]*len(psych_flex_CR[psych_flex_CR['MitreID'] ==  part]), ignore_index=True)
    df_part_long.reset_index(inplace=True)
    psych_flex_CR_demog_int = pd.concat([df_part_long, psych_flex_CR[psych_flex_CR['MitreID'] ==  part].reset_index()], axis = 1)
    psych_flex_CR_demog = psych_flex_CR_demog.append(psych_flex_CR_demog_int)

#confirm the two data tables are now the same lenght
print('psych_flex_CR and psych_flex_CR_demog are the same length:', psych_flex_CR.shape[0] == psych_flex_CR_demog.shape[0])
print(psych_flex_CR.shape[0])
print(psych_flex_CR_demog.shape[0])
print('does the math make sense?', psych_flex_CR_demog.shape[0] == (psych_flex_CR_demog['ID'].values == psych_flex_CR_demog['MitreID'].values).sum())
psych_flex_CR_demog.head()

In [ ]:
psych_flex_CR_demog.columns.values

In [ ]:
PF_CR_demog_features = ['MitreID', 'ParticipantID',
       'PrimaryUnit', 'SmartPhone', 'Sex', 'Shift', 'Wave', 'bornUS', 'lang', 'englyrs', 
                        'activity_num', 'pf_mgt', 'exp_neg', 'exp_pos',
       'exp_neut', 'ocb', 'inter.deviance', 'org.deviance', 'extraversion',
       'agreeableness', 'conscientiousness', 'neuroticism', 'openness',
       'psqi', 'fatigue', 'Stress', 
       'Flexibility', 'Inflexibility', 'Engagement', 'start_delay',
       'time_to_complete', 'wave_study_day', 'survey_count', 'day_of_week',
       'working', 'wave_study_date_bin', 'missing_values', 'uncompleted',
       'uncompleted_day_bin', 'longest_string_count',
       'longest_string_answer', 'longest_string_ratio',
       'longest_string_ratio_bin', 'bad_times_low', 'bad_times_high',
       'ls_7_ratio', 'ls_7_ratio_bin']

PF_CR_features = ['activity_num', 'pf_mgt', 'exp_neg', 'exp_pos',
       'exp_neut', 'day_of_week', 'working', 'start_delay',
       'time_to_complete', 'wave_study_date_bin', 'missing_values', 'uncompleted',
       'uncompleted_day_bin', 'longest_string_count',
       'longest_string_answer', 'longest_string_ratio',
       'longest_string_ratio_bin', 'bad_times_low', 'bad_times_high',
       'ls_7_ratio', 'ls_7_ratio_bin']

PF_CR_demog_df = psych_flex_CR_demog[PF_CR_demog_features]
PF_CR_df = psych_flex_CR_demog[PF_CR_features]

In [ ]:
PF_CR_demog_df.groupby('wave_study_date_bin').corr().to_csv('corr.csv')

In [ ]:
#use order from above
g = sns.catplot(x="wave_study_date_bin", y="longest_string_count", data=PF_CR_demog_df, kind="bar")
g.fig.set_size_inches(15,5)

In [ ]:
#use order from above
g = sns.catplot(x="wave_study_date_bin", y="longest_string_answer", data=PF_CR_demog_df, kind="bar")
g.fig.set_size_inches(15,5)

In [ ]:
#use order from above
g = sns.catplot(x="wave_study_date_bin", y="longest_string_ratio_bin", data=PF_CR_demog_df, kind="bar")
g.fig.set_size_inches(15,5)

In [ ]:
#use order from above
g = sns.catplot(x="wave_study_date_bin", y="ls_7_ratio_bin", data=PF_CR_demog_df, kind="bar")
g.fig.set_size_inches(15,5)

In [ ]:
#use order from above
g = sns.catplot(x="longest_string_count", y="bad_times_low", data=PF_CR_demog_df, kind="bar", col='wave_study_date_bin', col_wrap=3)
g.fig.set_size_inches(15,5)

In [ ]:
#use order from above
g = sns.catplot(x="longest_string_count", y="ls_7_ratio", data=PF_CR_demog_df, kind="bar", col='wave_study_date_bin', col_wrap=3)
g.fig.set_size_inches(15,5)

In [ ]:
PF_CR_demog_df['MitreID'].unique()

In [ ]:
unique_ids = PF_CR_demog_df['MitreID'].unique()
for u_id in unique_ids:
    g = sns.catplot(x="wave_study_day", y="longest_string_count", data=PF_CR_demog_df[PF_CR_demog_df['MitreID'] == u_id], kind="bar")
    g.fig.set_size_inches(10,5)
    #axes = g.axes
    plt.ylim([0,13])
    #g.set_ylim(0, 13)
    plt.show()

In [ ]:
#get participant ID order when sorted by longest string in the last time bin
wave_1_ls = PF_CR_demog_df[PF_CR_demog_df['wave_study_date_bin'] == PF_CR_demog_df['wave_study_date_bin'].unique()[-1]].sort_values('longest_string_count')
ls_index = wave_1_ls['MitreID'].unique()

In [ ]:
#use order from above
g = sns.catplot(x="MitreID", y="longest_string_count", data=PF_CR_demog_df, kind="bar", col='wave_study_date_bin', 
                ci=None, order=ls_index)
g.fig.set_size_inches(25,10)

In [ ]:
#use order from above
g = sns.catplot(x="MitreID", y="ls_7_ratio_bin", data=PF_CR_demog_df, kind="bar", col='wave_study_date_bin', 
                ci=None, order=ls_index)
g.fig.set_size_inches(25,10)

In [ ]:
day_bins = PF_CR_demog_df['wave_study_date_bin'].unique()
plt.figure(figsize=(25, 10))
ax = PF_CR_demog_df[PF_CR_demog_df['wave_study_date_bin'] == day_bins[0]].sort_values('longest_string_count').groupby(['ParticipantID'])['longest_string_count'].mean().plot(kind='bar')
#ax = PF_CR_demog_df[PF_CR_demog_df['wave_study_date_bin'] == day_bins[1]].groupby(['ParticipantID'])['time_to_complete'].mean().plot(kind='bar')
#ax = PF_CR_demog_df[PF_CR_demog_df['wave_study_date_bin'] == day_bins[2]].groupby(['ParticipantID'])['time_to_complete'].mean().plot(kind='bar')
#ax = PF_CR_demog_df[PF_CR_demog_df['wave_study_date_bin'] == day_bins[3]].groupby(['ParticipantID'])['time_to_complete'].mean().plot(kind='bar')
#ax = PF_CR_demog_df[PF_CR_demog_df['wave_study_date_bin'] == day_bins[4]].groupby(['ParticipantID'])['time_to_complete'].mean().plot(kind='bar')
plt.legend(('job', 'health', 'personality', 'psych_flex', 'engage_psycap'))
plt.title('Mean time to complete survey across time')
ax.set_ylabel('Mean time to complete survey')
plt.show

In [ ]:
#use order from above
g = sns.catplot(x="MitreID", y="longest_string_ratio_bin", data=PF_CR_demog_df, kind="bar", col='wave_study_date_bin', 
                ci=None, order=ls_index)
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.catplot(x="bad_times", y="longest_string_count", data=PF_CR_demog_df, kind="bar", col='wave_study_date_bin')
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.catplot(x="fatigue", y="longest_string_count", data=PF_CR_demog_df, kind="bar", col='wave_study_date_bin', order=['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100'])
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.FacetGrid(PF_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "ocb", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
g = sns.FacetGrid(PF_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "org.deviance", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
g = sns.FacetGrid(PF_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "inter.deviance", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
PF_CR_demog_df['Inflexibility'] = PF_CR_demog_df['Inflexibility'].astype(float)

In [ ]:
g = sns.FacetGrid(PF_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "Inflexibility", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
PF_CR_demog_df['Stress'] = PF_CR_demog_df['Stress'].astype(float)

In [ ]:
g = sns.FacetGrid(PF_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "Stress", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
#create CR summary feature (count of positive CR parameters)

counts = []

for index, row in psych_flex_only.iterrows():
    count = 0
    if row['missing_values'] > 0:
        count += 1
    #if row['uncompleted'] > 0:
        #count += 1
    if row['longest_string'] > 7:
        count += 1
    
    counts.append(count)

psych_flex_only['CR_count'] = counts

In [ ]:
#save to csv
psych_flex_only.to_csv('psych_flex_only_CR.csv')